# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Управление браузером с Selenium: пример работы  с ВКонтакте

### Часть 1: залогиниваемся во ВКонтакте

Импортируем необходимые компоненты Selenium:

In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

Импортируем функцию для запроса пароля, который скрывает вводимые с клавиатуры символы:

In [2]:
from getpass import getpass

Открываем браузер – начинаем новую сессию работы с Selenium:

In [3]:
br = wd.Chrome(executable_path='/Users/allat/Desktop/chromedriver')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


**NB.** На предупреждение можно не обращать внимание, если новое окно с браузером открывается.  В новой версии Selenium связь с драйвером обеспечивается не за счёт ссылки на исполняемый файл (`chromedriver` или `chromedriver.exe`), а через объект типа `Service`. Более новый код такой:

    from selenium import webdriver as wd
    from selenium.webdriver.chrome.service import Service
    
    serv = Service("/Users/allat/Desktop/chromedriver")
    br = wd.Chrome(service=serv)
    
Особой разницы нет, просто нужна ещё функция `Service()`, чтобы внутри неё прописать путь к драйверу.

Переходим в браузере на главную страницу ВКонтакте:

In [4]:
br.get("https://vk.com/")

Находим поле для логина:

In [5]:
login = br.find_element(By.ID, "index_email")

Отправляем туда свой логин:

In [6]:
login.send_keys("allatambov@mail.ru")

Чтобы не искать на странице кнопку *Войти* (не очень удобно), просто имитируем нажатие кнопки *Enter*:

In [7]:
login.send_keys(Keys.ENTER)

Находим поле для ввода пароля – тут надо постараться, потребуется просмотр кода страницы через интсрументы разработчика:

In [8]:
password = br.find_element(By.NAME, "password")

Запрашиваем пароль с клавиатуры у самих себя:

In [9]:
my_password = getpass()

········


Отправляем пароль в соответствующее поле и снова имитируем нажатие *Enter*:

In [10]:
password.send_keys(my_password)

In [11]:
password.send_keys(Keys.RETURN)

Готово! Можно искать информацию!

### Часть 2: ищем пользователей

Перейдём на страницу поиска друзей:

In [12]:
br.get("https://vk.com/friends")

Найдём на этой странице ссылку с текстом *Поиск друзей*, чтобы перейти к странице поиска пользователей по заданным критериям, и кликнем на неё:

In [13]:
search = br.find_element(By.LINK_TEXT, "Поиск друзей")
search.click() 

Чтобы формировать критерии для поиска, нам нужно открыть меню с фильтрами – развернуть меню *Параметры поиска*. Изучив исходный код страницы, видим, что параметры поиска можно найти по id:

In [14]:
pars = br.find_element(By.ID, "friends_filters_block")
pars.click()

Для начала выберем город. В исходном коде страницы поле для ввода города имеет тэг `<input>` и id, равный `cCity`:

In [15]:
city = br.find_element(By.ID, "cCity")

Объект `city` – это объект типа `webelement.WebElement`, то есть элемент страницы, с которым умеет работать модуль `webdriver` из `selenium`. По такому объекту тоже можно выполнять поиск с помощью метода `find_element`. 

В нашем случае внутри этого элемента нужно найти другой, с тэгом `<input>`, потому что нас интересует поле для ввода значения. Если этим шагом пренебречь и попытаться ввести название страны прямо в `city`, мы получим ошибку вида `element not interactable`, потому что сам по себе раздел со страной никакого взаимодействия с пользователем не предполагает, его нельзя редактировать, на него нельзя кликать и прочее.

Поэтому найдём внутри `city` поле для ввода значения по тэгу:

In [16]:
city_inp = city.find_element(By.TAG_NAME, "input")
city_inp.send_keys("Москва") 

Отлично! Значение выбрано. Но есть проблема – оно «повисло» в воздухе, опция с выбором страны отображается как выбранная в выпадающем меню, но в самом поле выбор не зафиксирован. Чтобы подтвердить выбор, нужно нажать на клавишу *Enter*:

In [17]:
br.implicitly_wait(3)  # задержка 3 секунды
city_inp.send_keys(Keys.RETURN)

А вот с полом всё поинтереснее: найти поле для ввода пола просто, а вот значения нужно выбирать, нажимая на радиокнопки (*radiobuttons*). Сначала найдём поле для выбора пола:

In [18]:
sex = br.find_element(By.ID, "cSex")

А теперь – все опции внутри (согласно исходному коду, они имеют тэг `<div>`):

In [19]:
values = sex.find_elements(By.TAG_NAME, "div")
values

[<selenium.webdriver.remote.webelement.WebElement (session="1d2ef91e3a5ed30c928847779b35d788", element="a9273d22-507c-464c-8c86-b9eefb3e5718")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1d2ef91e3a5ed30c928847779b35d788", element="f8c77b0f-e428-4266-ab2b-a15d0a21cc89")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1d2ef91e3a5ed30c928847779b35d788", element="70f9e542-e470-433a-a188-5435b753e061")>]

Обратите внимание: здесь метод `find_elements()`, не `find_element()`, потому что результатов ожидается несколько. Методы вида `find_element()` возвращают только первое совпадение на странице, методы вида `find_elements()` – все совпадения на странице (можно провести аналогию с `find` и `find_all` в `BeautifulSoup`).

Выбираем мужской пол – это второй элемент списка – и кликаем на него:

In [20]:
values[1].click()

На этом закончим работу с фильтрами и перейдём к результатам. На этом этапе возможности `selenium` нам пока не понадобятся, нам нужно только запросить исходный код страницы, которая сейчас открыта в окне браузера, управляемом из Python, и продолжить работу с HTML с помощью BeautfulSoup.

In [21]:
html = br.page_source

### Часть 3: обработка результатов

In [22]:
from bs4 import BeautifulSoup

In [23]:
soup = BeautifulSoup(html)

### Задача 1

Создайте список `divs` с фрагментами html-кода, каждый из которых соответствует одной «карточке» пользователя с именем и ссылкой на профиль. 

In [28]:
divs = soup.find_all("div", {"class" : "labeled name"})

### Задача 2

Напишите функцию `get_person()` для извлечения имени и ссылки на профиль пользователя (ссылки должны быть полными) и примените её ко всем пользователям на странице. Сохраните результаты в датафрейм `people`.

In [29]:
# изучаем один элемент divs
d = divs[0]
d

<div class="labeled name"><a href="/birdnice" onclick="return nav.go(this, event);">Дмитрий Сорокин</a></div>

In [31]:
# извлекаем имя и ссылку на профиль

name = d.text
href = d.find("a")["href"]
print(name, href)

Дмитрий Сорокин /birdnice


In [32]:
# пишем функцию – ссылки делаем полными

def get_person(d):
    name = d.text
    href = d.find("a")["href"]
    full_href = "https://vk.com" + href
    return name, full_href

In [33]:
# применяем функцию

L = [get_person(d) for d in divs] 
L

[('Дмитрий Сорокин', 'https://vk.com/birdnice'),
 ('Никита Зарипов', 'https://vk.com/letsgowithyou'),
 ('Мишаня Аникин', 'https://vk.com/muaymikethai'),
 ('Иоанн Довгополый', 'https://vk.com/dereinzigemitdemeigentum'),
 ('Алексей Непрокин', 'https://vk.com/lex_1979'),
 ('Степан Кyзнeцoв', 'https://vk.com/id104687424'),
 ('Артём Черняев-Ермоленко', 'https://vk.com/tatorhe'),
 ('Егор Гладких', 'https://vk.com/yegomaru'),
 ('Илья Иншаков', 'https://vk.com/id136161093'),
 ('Дан Подлецкий', 'https://vk.com/dan_podletsky'),
 ('Даниил Аксёнов', 'https://vk.com/danhse'),
 ('Артем Терещенков', 'https://vk.com/tereshchka'),
 ('Михаил Садов', 'https://vk.com/m_sadov'),
 ('Азиз Тулаганов', 'https://vk.com/aziz_tulaganov'),
 ('Никита Радченко', 'https://vk.com/id493903621'),
 ('Александр Усов', 'https://vk.com/id6084115'),
 ('Иван Александров', 'https://vk.com/halarge'),
 ('Сергей Михайлов', 'https://vk.com/samikhaylov21'),
 ('Евгений Лямин', 'https://vk.com/nemozhenya'),
 ('Кирилл Меньшиков', 'htt

In [34]:
# превращаем в датафрейм

import pandas as pd
people = pd.DataFrame(L)
people.columns = ["username", "userlink"]

In [35]:
people.head()

,username,userlink
0,Дмитрий Сорокин,https://vk.com/birdnice
1,Никита Зарипов,https://vk.com/letsgowithyou
2,Мишаня Аникин,https://vk.com/muaymikethai
3,Иоанн Довгополый,https://vk.com/dereinzigemitdemeigentum
4,Алексей Непрокин,https://vk.com/lex_1979
